# Notes

**Important**

This notebook was created on *April 23, 2024, at 19:08*  with the intention of providing detailed explanations and interpretations of the processes involved. While Python scripts are powerful, they can sometimes be complex and require a significant investment in terms of analysis and understanding. This document aims to simplify this task by breaking down and explaining each code segment in detail.

In this notebook, we will undertake to revisit and detail the implementations of the MNIST dataset, a dataset widely used in the machine learning community, initially popularized by Yann LeCun. The goal is to implement two types of neural architectures: a Multi-Layer Perceptron (MLP) and a Convolutional Neural Network (CNN). We will then compare their performances through various metrics.

Additionally, special attention will be given to visualizing backpropagation to identify the global optimum. This approach aims to illustrate not only how modifications to the weights affect model accuracy during training but also to demonstrate optimization dynamics in action.

This work is essential for those looking to deepen their understanding of deep learning fundamentals as well as for those wishing to refine their ability to optimize machine learning algorithms.

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## PyTorch
import torch 
import torchvision

In [2]:
# Load data
train_data = torchvision.datasets.MNIST(root='data', train=True,
                                        download=True, transform=torchvision.transforms.ToTensor())

test_data = torchvision.datasets.MNIST(root='data', train=False,
                                       download=True, transform=torchvision.transforms.ToTensor())

print('Data are ready!')

Data are ready!


We have downloaded the data, now to extract the images and labels we will type the following command:

```bash
X = train_data.data
y = train_data.targets
```

In [17]:
# Examples
# Images are in uint8
print(f'The label ==> {train_data.targets[0]}')
train_data.data[0]

The label ==> 5


tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
          18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0],
        [  0,   0,   0,   0,   0,   0,   0,   

## Preparation

We are going to prepare the images here. The images are already in an appropriate format and require no special preparation other than normalization. As you may notice, the pixel values range from 0 to 255. This range can increase the computational load and introduce significant variance. Therefore, it is crucial to normalize the data. Using a MinMaxScaler normalization could be beneficial to scale our data from 0 to 1.

The MinMax normalization formula is given by: $MinMax = \frac{X - X_{\text{min}}}{X_{\text{max}} - X_{\text{min}}}$.

Next, we will load these normalized data into a DataLoader, which will compact our data and organize it into batches. This will enable us to use the GPU for computations, making the process more efficient.

In [ ]:
def plot_images_grid(data_loader, num_images=25, title=None):
    """
    Plots a grid of images with labels.

    Args:
    data_loader (torch.utils.data.DataLoader): DataLoader for the dataset.
    num_images (int): Number of images to display in the grid.
    title (str): Title of the plot.
    """
    # Fetch a batch of images and labels
    images, labels = next(iter(data_loader))
    
    # Make sure we only plot the specified number of images
    images = images[:num_images]
    labels = labels[:num_images]

    # Create a grid of images
    img_grid = torchvision.utils.make_grid(images, nrow=int(num_images ** 0.5), normalize=True)

    # Show images
    plt.figure(figsize=(10, 10))
    plt.imshow(img_grid.permute(1, 2, 0))  # Rearrange the order of channels
    plt.axis('off')
    
    # Adding the title
    if title:
        plt.title(title)

    # Show labels in the title or a legend (optional)
    labels_str = ', '.join(map(str, labels.tolist()))
    plt.xlabel(f"Labels: {labels_str}")

    plt.show()
